# 교차검증 (cross-validation)
  
  
train_test_split의 내용을 보면 과적합(Overfitting)에 취약한 약점을 가질 수 있다.  
과적합은 모델이 학습 데이터에만 과도하게 최적화되어, 실제 예측을 다른 데이터로 수행할 경우에는 예측 성능이 과도하게 떨어지는 것을 말한다.   
고정된 학습 데이터와 테스트 데이터로 평가를 하다 보면 테스트 데이터에만 최적의 성능을 발휘할 수 있도록 편향되게 모델을 유도하는 경향이 생기게 된다.   
결국은 해당 테스트 데이터에만 과적합되는 학습 모델이 만들어져 다른 테스트용 데이터가 들어올 경우에는 성능이 저하된다.   
이러한 문제점을 개선하기 위해 교차 검증을 이용해 더 다양한 학습과 평가를 수행한다. 

# K 폴드 교차 검증

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

In [4]:
iris = load_iris()

features = iris.data
label = iris.target

dt_clf = DecisionTreeClassifier(random_state=156)

kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기 : ', features.shape[0])

붓꽃 데이터 세트 크기 :  150


전체 붓꽃 데이터는 모두 150개이므로, 학습용 데이터는 4/5인 120개, 검증 테스트 데이터 세트는 1/5인 30개로 분할된다.  

In [10]:
n_iter = 0

for train_index, test_index in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('{0} 교차 검증 정확도 : {1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}'.format(n_iter, accuracy, train_size, test_size))
    print('{0} 검증 세트 인덱스 : {1}'.format(n_iter, test_index))
    print()
    cv_accuracy.append(accuracy)

print()
print('평균 검증 정확도 : ', np.mean(cv_accuracy))

1 교차 검증 정확도 : 1.0, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
1 검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

2 교차 검증 정확도 : 0.9667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
2 검증 세트 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

3 교차 검증 정확도 : 0.8667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
3 검증 세트 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

4 교차 검증 정확도 : 0.9333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
4 검증 세트 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

5 교차 검증 정확도 : 0.7333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
5 검증 세트 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]


평균 검증 정확도 :  0.9


# Stratified K 폴드

불균형한 분포도를 가진 레이블 데이터 집합을 위한 K 폴드 방식이다.   
불균형한 분포도라는 의미는 특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 것을 말한다.  
대출 사기(1) 레이블이 0.001% 분포해 있을 때 건수는 작지만 알고리즘이 대출 사기를 예측하기 위한 중요한 피처 값을 가지고 있기 때문에 매우 중요하다.  
이처럼 K 폴드가 학습 및 테스트 세트에 제대로 분배하지 못하는 경우의 문제를 해결해줄 수 있다. 

In [11]:
import pandas as pd

In [12]:
iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

레이블 값의 분포도를 먼저 확인한다.

In [13]:
kfold = KFold(n_splits=3)

n_iter = 0
for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    
    print('교차 검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('검증 레이블 데이터 분포 : \n', label_test.value_counts())

교차 검증 : 1
학습 레이블 데이터 분포 : 
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 0    50
Name: label, dtype: int64
교차 검증 : 2
학습 레이블 데이터 분포 : 
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 1    50
Name: label, dtype: int64
교차 검증 : 3
학습 레이블 데이터 분포 : 
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    50
Name: label, dtype: int64


첫 번째 교차 검증을 살펴보자.  
  
학습 레이블은 1,2 밖에 없으므로 0의 경우는 전혀 학습하지 못한다.   
반대로 검증 레이블은 0밖에 없으므로 학습 모델은 절대 0을 예측하지 못한다.  
이런 유형으로 교차 검증 데이터 세트를 분할하면 검증 예측 정확도는 0이 될 수 밖에 없다.  
이러한 문제를 해결하기 위해서 나올 알고리즘이 Stratified K 폴드 알고리즘이다.

In [15]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    
    print('교차 검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('검증 레이블 데이터 분포 : \n', label_test.value_counts())

교차 검증 : 1
학습 레이블 데이터 분포 : 
 2    33
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    17
1    17
0    17
Name: label, dtype: int64
교차 검증 : 2
학습 레이블 데이터 분포 : 
 2    33
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    17
1    17
0    17
Name: label, dtype: int64
교차 검증 : 3
학습 레이블 데이터 분포 : 
 2    34
1    34
0    34
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    16
1    16
0    16
Name: label, dtype: int64


출력 결과를 살펴보면 학습 및 검증 레이블 데이터 분포가 각각 균일하게 이루어졌음을 알 수 있다.   
이렇게 분할이 되어야 레이블 값 0,1,2 를 모두 학습할 수 있고, 이에 기반해 검증을 수행할 수 있다. 

다음은 붓꽃 데이터를 이용하여 교차 검증을 진행해보도록 한다.

In [19]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy = []

for train_index, test_index in skfold.split(features, label):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('{0} 교차 검증 정확도 : {1}'.format(n_iter, test_index))
    print('{0} 검증 세트 인덱스 : {1}'.format(n_iter, test_index))
    print()
    cv_accuracy.append(accuracy)
    print('교차 검증별 정확도 : ', np.round(cv_accuracy, 4))
    print('평균 검증 정확도 : ', np.mean(cv_accuracy))
    print()

1 교차 검증 정확도 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116]
1 검증 세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116]

교차 검증별 정확도 :  [0.9804]
평균 검증 정확도 :  0.9804

2 교차 검증 정확도 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133]
2 검증 세트 인덱스 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133]

교차 검증별 정확도 :  [0.9804 0.9216]
평균 검증 정확도 :  0.9510000000000001

3 교

일반적으로 분류에서의 교차 검증은 k 폴드가 아니라 Stratified K폴드로 분할되어야 한다.  
회귀에서는 Stratified K폴드가 지원되지 않는데, 이유는 회귀의 결정값은 이산값 형태의 레이블이 아니라  
연속된 숫자값이기 때문에 결정값별로 분포를 정하는 의미가 없기 때문이다. 